In [1]:
import pandas as pd

from bs4 import BeautifulSoup

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from collections import defaultdict

In [2]:
from gensim import models
import numpy as np

In [3]:
import gensim
from gensim import corpora
from pprint import pprint
import string

In [4]:
df = pd.read_csv(r"D:\data\openparliament\statements_nospeaker_en.csv")

C:\Users\alecr\Anaconda3\envs\gensim_env\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (20,23,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.drop('Unnamed: 0',axis=1, inplace=True)

In [6]:
df.head() 

,id,politician_id,riding_id,party_id,end_date,start_date,id.1,document_id,time,h1_en,...,slug,urlcache,h1_fr,h2_fr,h3_fr,who_fr,who_context_fr,wordcount_en,tokenized_content,slug_length
0,2611,3465,35066,4,2005-11-29,1997-09-22,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
1,3210,3465,70224,4,1997-04-27,1994-01-17,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
2,4305,173,70358,1,\N,2015-10-19,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
3,1534,173,47007,1,2015-10-19,2004-10-04,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
4,1541,43,47014,1,2015-10-19,2004-10-04,313253,1621,2004-02-06 10:05:00-05,NaN,...,garry-breitkreuz-1,/debates/2004/2/6/garry-breitkreuz-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'today', ...",3


In [8]:
stop_words = set(stopwords.words('english'))

In [9]:
punctuation = '!"#$%&()*+,-./:;<=>@[]^_`{|}~' + "''"

In [10]:
pun_trans = str.maketrans(dict.fromkeys(punctuation, ' '))

In [11]:
ps = PorterStemmer() 

In [12]:
content_en_list = []  # append the content from each chunk df here 

In [13]:
def tokenize_column (row):
    soup = BeautifulSoup(row)
    text = soup.text.lower()
    text = text.translate(pun_trans)
    text = text.replace('\\n', ' ')
    text = text.replace('—', ' ')
    words = word_tokenize(text)
    return words

In [14]:
stop_words = set(stopwords.words('english'))

In [15]:
def remove_stopwords(word_tokens):
    tokens = [w for w in word_tokens if not w in stop_words]
    tokens = [t for t in tokens if not t.isdigit()]
    words = [ps.stem(w) for w in tokens]
    return words

In [16]:
small_df = df[:100]

In [17]:
df.columns

Index(['id', 'politician_id', 'riding_id', 'party_id', 'end_date',
       'start_date', 'id.1', 'document_id', 'time', 'h1_en', 'h2_en',
       'member_id', 'who_en', 'content_en', 'sequence_en', 'wordcount',
       'politician_id.1', 'procedural', 'h3_en', 'who_hocid', 'content_fr',
       'statement_type', 'written_question', 'source_id', 'who_context_en',
       'slug', 'urlcache', 'h1_fr', 'h2_fr', 'h3_fr', 'who_fr',
       'who_context_fr', 'wordcount_en', 'tokenized_content', 'slug_length'],
      dtype='object')

In [26]:
politicians = list(set(df.politician_id))

In [27]:
vocab_count_dict = defaultdict()

In [28]:
def add(k,v_list):
    if k in vocab_count_dict:
        for v in v_list:
            vocab_count_dict[k].add(v)
    else:
        for v in v_list:
            vocab_count_dict[k] = {v}

In [29]:
%%time
max_tokens = 0
count = 0

for index, row in df.iterrows(): 
    soup = BeautifulSoup(row.content_en)
    text = soup.text.lower()
    text = text.translate(pun_trans)
    text = text.replace('\\n', ' ')
    text = text.replace('—', ' ')
    word_tokens = word_tokenize(text)
    tokens = [w for w in word_tokens if not w in stop_words]
    tokens = [t for t in tokens if not t.isdigit()]
    words = [ps.stem(w) for w in tokens]
    
    pid = row['politician_id']
    add(pid,words)
    
    if len(words)>max_tokens:
        max_tokens = len(words)
    if count%10000==0:
        print(count)
        print('updating id', row['politician_id'], words[:10])
    
    
    count+=1
#print(row.who_en, "gave the longest speech at", max_tokens, 'words')
    
        

0
updating id 3465 ['mr', 'speaker', 'rise', 'point', 'order', 'believ', 'would', 'find', 'unanim', 'consent']
10000
updating id 415 ['mr', 'speaker', 'ever', 'sinc', 'prime', 'minist', 'fled', 'davo', 'switzerland', 'announc']
20000
updating id 554 ['madam', 'speaker', 'thought', 'red', 'bait', 'well', 'end', 'time', 'ago', 'guess']
30000
updating id 43 ['mr', 'speaker', 'know', 'member', 'tri', 'divert', 'attent', 'away', 'discuss', 'today']
40000
updating id 5241 ['mr', 'speaker', 'keep', 'give', 'answer', 'tell', 'truth', 'give', 'untruth', 'answer']
50000
updating id 405 ['mr', 'speaker', 'petit', 'numer', 'canadian', 'across', 'countri', 'state', 'undersign', 'citizen']
60000
updating id 287 ['move', 'bill', 'read', 'third', 'time', 'pass']
70000
updating id 285 ['mr', 'speaker', 'hon', 'member', 'accus', 'read', 'obvious', 'listen', 'one', 'cornerston']
80000
updating id 414 ['mr', 'speaker', 'made', 'document', 'legal', 'capabl', 'made', 'given', 'committe', 'given']
90000
upda

720000
updating id 10542 ['mr', 'speaker', 'problem', 'current', 'affect', 'public', 'servic', 'pay', 'system', 'total']
730000
updating id 9770 ['mr', 'speaker', 'countri', 'face', 'public', 'health', 'crisi', 'crisi', 'opioid', 'overdos']
740000
updating id 9899 ['mr', 'speaker', 'rise', 'speak', 'opposit', 'day', 'motion', 'tabl', 'member', 'cypress']
750000
updating id 10040 ['mr', 'speaker', 'would', 'like', 'say', 'good', 'even', 'minist', 'thank', 'us']
760000
updating id 216 ['mr', 'speaker', 'want', 'tell', 'member', 'cheerlead', 'pmo', 'tell', 'exactli', 'fact']
770000
updating id 404 ['mr', 'speaker', 'minist', 'financ', 'claim', 'listen', 'busi', 'owner', 'across', 'countri']
780000
updating id 10162 ['mr', 'speaker', 'govern', 'beneath', 'contempt', 'davi', 'shipyard', 'situat', 'quebec', 'citi']
790000
updating id 8678 ['mr', 'speaker', 'govern', 'member', 'given', 'explan', 'colleagu', 'say', 'independ', 'non']
800000
updating id 10214 ['mr', 'chair', 'follow', 'colleagu

# loading defaultdict from csv

In [40]:
df[df.politician_id == 3465]

,id,politician_id,riding_id,party_id,end_date,start_date,id.1,document_id,time,h1_en,...,slug,urlcache,h1_fr,h2_fr,h3_fr,who_fr,who_context_fr,wordcount_en,tokenized_content,slug_length
0,2611,3465,35066,4,2005-11-29,1997-09-22,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
1,3210,3465,70224,4,1997-04-27,1994-01-17,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
51,2611,3465,35066,4,2005-11-29,1997-09-22,268849,2026,2002-06-06 22:50:00-04,Government Orders,...,marlene-catterall-6,/debates/2002/6/6/marlene-catterall-6/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'would', 'ask', '...",3
52,3210,3465,70224,4,1997-04-27,1994-01-17,268849,2026,2002-06-06 22:50:00-04,Government Orders,...,marlene-catterall-6,/debates/2002/6/6/marlene-catterall-6/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'would', 'ask', '...",3
138,2611,3465,35066,4,2005-11-29,1997-09-22,261995,2000,2002-04-23 18:15:00-04,Government Orders,...,marlene-catterall-5,/debates/2002/4/23/marlene-catterall-5/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'think', 'you', '...",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526615,3210,3465,70224,4,1997-04-27,1994-01-17,233241,1881,2001-05-08 17:50:00-04,Government Orders,...,marlene-catterall-3,/debates/2001/5/8/marlene-catterall-3/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'believe', 'you',...",3
527159,2611,3465,35066,4,2005-11-29,1997-09-22,233261,1881,2001-05-08 17:55:00-04,Government Orders,...,marlene-catterall-5,/debates/2001/5/8/marlene-catterall-5/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'think', 'you', '...",3
527160,3210,3465,70224,4,1997-04-27,1994-01-17,233261,1881,2001-05-08 17:55:00-04,Government Orders,...,marlene-catterall-5,/debates/2001/5/8/marlene-catterall-5/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'think', 'you', '...",3
527247,2611,3465,35066,4,2005-11-29,1997-09-22,233251,1881,2001-05-08 17:50:00-04,Government Orders,...,marlene-catterall-4,/debates/2001/5/8/marlene-catterall-4/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'you', 'will', 'find',...",3


In [7]:
import pickle

In [9]:
with open(r"D:\data\openparliament\models\vocab_count_dict.pkl", "rb") as f:
    vocab_count_dict = pickle.load(f)